In [1]:
pip install adabelief-pytorch==0.1.0

In [2]:
pip install adabound

In [3]:
pip install torch_optimizer

     |████████████████████████████████| 71kB 6.1MB/s 


In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch_optimizer as optset
import torch.nn as nn
from adabelief_pytorch import AdaBelief
import adabound

In [5]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.RandomResizedCrop(224), 
                                      transforms.RandomHorizontalFlip(), 
                                      transforms.ToTensor(),
                                      normalize])
test_transform = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     normalize])

In [6]:
batch_size = 100
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=train_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [7]:
import time

def timeSince(since):
    now = time.time()
    s = now - since
    m = s // 60
    s -= m * 60
    return '%dm %ds' % (m, s)

In [8]:
import torchvision.models as models

optimizer_names = ['AdaBelief', 'AdaBound', 'Adam', 'LAMB', 'SGD', 'Yogi']

In [9]:
def evaluateImageModel(model, data, criterion, device):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for image, labels in data:
            image, labels = image.to(device), labels.to(device)
            output = model(image)
            pred = torch.argmax(output, dim=1)
            correct += (pred == labels).sum().item()
            total += len(labels)
            loss = criterion(output, labels)
            total_loss += loss.item()

    return total_loss / len(data), 1 - correct / total

In [10]:
def trainImageModel(model, train_data, val_data, n_epochs, criterion, optimizer, device, path):
    model.train()
    train_error_list = []
    val_error_list = []
    min_error = None
    step = 0
    print_every = len(train_data)
    start = time.time()
    for epoch in range(n_epochs):
        running_loss = 0.0
        running_correct = 0
        running_total = 0
        for image, labels in train_data:
            optimizer.zero_grad()
            step += 1
            image, labels = image.to(device), labels.to(device)
            output = model(image)
            pred = torch.argmax(output, dim=1)
            running_correct += (pred == labels).sum().item()
            running_total += len(labels)
            loss = criterion(output, labels)
            running_loss += loss.item()
            loss.backward()
            optimizer.step()
            if step % print_every == 0:
                val_loss, val_error = evaluateImageModel(model, val_data, criterion, device)
                print(('%d/%d (%s) train loss: %.3f, train error: %.2f%%, val loss: %.3f, val error: %.2f%%') %
                      (epoch + 1, n_epochs, timeSince(start), running_loss / len(train_data), 
                       100 *(1 - running_correct / running_total), val_loss, 100 * val_error))
                train_error_list.append(1 - running_correct / running_total)
                val_error_list.append(val_error)
                if min_error is None or min_error > val_error:
                    if min_error is None:
                        print(('Validation error rate in first epoch: %.2f%%') % (100 * val_error))
                    else:
                        print(('Validation error rate is decreasing: %.2f%% --> %.2f%%') % 
                              (100 * min_error, 100 * val_error))
                    min_error = val_error
                    print('Saving model...')
                    torch.save(model, path)
                
                model.train()
                running_loss = 0.0
                running_correct = 0
                running_total = 0
    
    return train_error_list, val_error_list

In [12]:
# Adabelief
model = models.resnet34()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.CrossEntropyLoss()
n_epochs = 50
adabelief_Res_train_list = None
adabelief_Res_test_list = None
optimizer = AdaBelief(model.parameters())
model.to(device)
path = 'AdabeliefRes.pt'
adabelief_Res_train_list, adabelief_Res_test_list = trainImageModel(model, trainloader, testloader,
                                                                    n_epochs, criterion, optimizer,
                                                                    device, path)
torch.save(adabelief_Res_train_list, 'adabeliefres_train.pt')
torch.save(adabelief_Res_test_list, 'adabeliefres_test.pt')

Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
Current version (0.1.0)  1e-16  True               True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer

Weight decoupling enabled in AdaBelief
Rectification enabled in AdaBelief
1/50 (1m 48s) train loss: 1.939, train error: 64.68%, val loss: 1.426, val error: 52.60%
Validation error rate in first epoch: 52.60%
Saving model...
2/50 (3m 38s) train loss: 1.416, train error: 51.12%, val loss: 1.769, val error: 50.84%
Validation error rate is decreasing: 52.60% --> 50.84%
Saving model...
3/50 (5m 29s) train loss: 1.214, train error: 43.14%, val loss: 1.036, val error: 35.59%
Validation error rate is decreasing: 50.84% --> 35.59%
Savi

In [13]:
from google.colab import files

files.download('adabeliefres_train.pt')
files.download('adabeliefres_test.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
train_list = None
test_list = None
model = models.resnet34()
optimizer = adabound.AdaBound(model.parameters())
model.to(device)
path = 'AdaBoundRes.pt'
train_list, test_list = trainImageModel(model, trainloader, testloader,
                                                                    n_epochs, criterion, optimizer,
                                                                    device, path)
torch.save(train_list, 'adaboundres_train.pt')
torch.save(test_list, 'adaboundres_test.pt')
files.download('adaboundres_train.pt')
files.download('adaboundres_test.pt')

1/50 (1m 48s) train loss: 1.845, train error: 67.37%, val loss: 1.489, val error: 56.35%
Validation error rate in first epoch: 56.35%
Saving model...
2/50 (3m 37s) train loss: 1.597, train error: 58.61%, val loss: 1.448, val error: 52.70%
Validation error rate is decreasing: 56.35% --> 52.70%
Saving model...
3/50 (5m 25s) train loss: 1.418, train error: 51.35%, val loss: 1.201, val error: 42.55%
Validation error rate is decreasing: 52.70% --> 42.55%
Saving model...
4/50 (7m 13s) train loss: 1.270, train error: 45.51%, val loss: 0.969, val error: 34.88%
Validation error rate is decreasing: 42.55% --> 34.88%
Saving model...
5/50 (9m 2s) train loss: 1.158, train error: 41.24%, val loss: 0.929, val error: 32.83%
Validation error rate is decreasing: 34.88% --> 32.83%
Saving model...
6/50 (10m 52s) train loss: 1.089, train error: 38.79%, val loss: 0.800, val error: 27.32%
Validation error rate is decreasing: 32.83% --> 27.32%
Saving model...
7/50 (12m 39s) train loss: 1.016, train error: 35.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
train_list = None
test_list = None
model = models.resnet34()
optimizer = optim.Adam(model.parameters())
model.to(device)
path = 'AdamRes.pt'
train_list, test_list = trainImageModel(model, trainloader, testloader, n_epochs, criterion, optimizer, device, path)
torch.save(train_list, 'adamres_train.pt')
torch.save(test_list, 'adamres_test.pt')
files.download('adamres_train.pt')
files.download('adamres_test.pt')

1/50 (1m 40s) train loss: 1.867, train error: 68.05%, val loss: 1.466, val error: 54.46%
Validation error rate in first epoch: 54.46%
Saving model...
2/50 (3m 21s) train loss: 1.482, train error: 53.75%, val loss: 1.271, val error: 44.34%
Validation error rate is decreasing: 54.46% --> 44.34%
Saving model...
3/50 (5m 3s) train loss: 1.256, train error: 44.81%, val loss: 1.081, val error: 36.12%
Validation error rate is decreasing: 44.34% --> 36.12%
Saving model...
4/50 (6m 45s) train loss: 1.118, train error: 39.85%, val loss: 1.027, val error: 34.24%
Validation error rate is decreasing: 36.12% --> 34.24%
Saving model...
5/50 (8m 27s) train loss: 1.018, train error: 35.85%, val loss: 0.746, val error: 25.74%
Validation error rate is decreasing: 34.24% --> 25.74%
Saving model...
6/50 (10m 9s) train loss: 0.948, train error: 33.23%, val loss: 0.710, val error: 24.25%
Validation error rate is decreasing: 25.74% --> 24.25%
Saving model...
7/50 (11m 50s) train loss: 0.881, train error: 30.9

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
path_list = ['adabeliefres_train.pt', 'adabeliefres_test.pt',
             'adaboundres_train.pt', 'adaboundres_test.pt',
             'adamres_train.pt', 'adamres_test.pt']

for path in path_list:
    files.download(path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>